<a href="https://colab.research.google.com/github/matthewbegun/MXN500/blob/main/MXN500_2024_LEC_11_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MXN500 Lecture 11 - And now for some more linear models!


In [ ]:
#@title Imports
if (!require(pacman)) install.packages("pacman")
pacman::p_load(tidyverse, broom, GGally)

In [ ]:
#@title Options
options(repr.plot.width=15, repr.plot.height=5, repr.plot.pointsize=24)

## Slide 11 - Correlation in Action

1. What is the correlation between petal length and petal width in the `iris` data set?
1. What do you expect the $𝑅^2$ value to be if you fit a linear model using petal width as the outcome variable, based on your result in above?
1. Which physio-chemical water quality variables in the `Ecologyv2.csv` data set are related to taxon richness?


In [ ]:
# What is the correlation between petal length and petal width in the iris data set?
cor(iris$Petal.Length, iris$Petal.Width)

In [ ]:
# What do you expect the  R2  value to be if you fit a linear model using petal width as the outcome variable, based on your result in above?
cor(iris$Petal.Length, iris$Petal.Width)^2

In [ ]:
# Can confirm this:
summary(lm(data=iris, Petal.Width ~ Petal.Length))

In [ ]:
# Which physio-chemical water quality variables in the Ecologyv2.csv data set are related to taxon richness?
eco <- read_csv("Ecologyv2.csv")
eco_vars <- c("Richness", "Turbidity", "DO", "Cond", "pH", "Temp")
eco_pairs <- ggpairs(eco, columns = eco_vars,
                     lower=list(continuous="smooth"),
                     diag=list(continuous="densityDiag"))
eco_pairs


We can see the following things about correlation with taxon richness:
- Strong negative correlation with turbidity.
- Moderately positive correlation with dissolved oxygen.
- Strongly negative correlation with conductivity (perhaps in groups).
- Weakly positive correlation with pH (not significant).
- Moderately negative correlation with temperature.


In [ ]:
# Table of correlation values:
eco_cor <- eco %>%
  select(all_of(eco_vars)) %>%
  cor() %>%
  round(., 3)
eco_cor

In [ ]:
# Pretty picture:
cor_long <- eco_cor %>%
  as.data.frame() %>%
  mutate(X = row.names(.)) %>%
  pivot_longer(cols = -X, names_to = "Y",
               values_to = "corr_value") %>%
  mutate(X = fct_relevel(X, eco_vars),
         Y = fct_relevel(Y, eco_vars))
eco_cor_heat <- ggplot(data=cor_long,
                       aes(x=X, y=Y)) +
  geom_tile(aes(fill=corr_value)) +
  scale_fill_gradient2(low="red", mid="white",
                       high="blue", midpoint = 0, limits=c(-1,1)) +
  theme(axis.text.x=element_text(angle=90, hjust=1, vjust=0.5)) +
  coord_equal() + xlab("") + ylab("")
eco_cor_heat

## Slide 15 -Analysis Roadmap

The `hills` data set from the `MASS` library contains record times in 1984 for 35 Scottish hill races.
- Distance in miles (on the map), dist.
- Total height gained during the route (in feet), climb.
- Record time (minutes), time.

1. Can we predict a record time based on the distance and the height gained during the race? Answer this question by following the given analysis roadmap.


### 1. Visualise data – exploratory analysis.

In [ ]:
# load the hills data
hills <- MASS::hills

# Visually summarise the data so that the reader understands what it is.
# Visualise all variables separately.
hills_pairs <- ggpairs(hills,
                       lower=list(continuous="smooth"),
                       diag=list(continuous="densityDiag"))
hills_pairs

In [ ]:
# Produce some numerical summaries.
hills %>%
  pivot_longer(c(dist, climb,  time), names_to = "variable", values_to = "value")  %>%
  group_by(variable) %>%
  summarise(Mean = mean(value),
            SD = sd(value),
            Min = min(value),
            Max = max(value))

### 2. Fit linear model.


- Estimate parameters.
- Calculate confidence intervals to summarise parameter uncertainty.
- Perform 𝑡 tests to check necessity of parameters.

In [ ]:
# Model specifying that time varies with dist and climb together.
hills_fit <- lm(data = hills, time ~ dist + climb)
hills_fit_tidy <- tidy(hills_fit, conf.int = TRUE)
hills_fit_tidy

For each parameter in our model, we want to know whether it is significantly different from zero with $\alpha=0.05$.

In this instance, we have evidence to reject the null hypothesis for each parameter.

Also have evidence to reject `intercept = 0`, but this is not as important, given that the model isn't feasible when `dist = 0` and `climb = 0`.


### 3. Assess goodness of fit.


In [ ]:
# How well did our model explain the variability in the data?
# Let's look at our R-squared value.
glance(hills_fit)$r.squared



In [ ]:
# If we had have fit our two models for dist and climb separately, we'd have:
lm(data = hills, time ~ dist) %>% glance() %>% select(r.squared)
lm(data = hills, time ~ climb) %>% glance() %>% select(r.squared)


 Remember: R_squared isn't just the sum of the R-squared values of these two models.

 Also, we can't treat R-squared as the square of sample correlation here, as correlation is only defined for two variables but here we have three.

 Other metrics of goodness of fit are also related to model assumptions.

### 4. Check model assumptions.


- Residuals analysis.


In [ ]:
# use fortify to get the residuals
hills_fit_fort <- fortify(hills_fit)

# plot residuals vs fitted values
ggplot(data=hills_fit_fort, aes(x=.fitted, y=.resid)) +
  geom_point() + xlab("Fitted") +
  geom_smooth() +
  ylab("Residuals = Observed - Fitted")

Some reasons for caution:
- Appears to be an outlier (top left).
- More variability for longer times.
- Mid sections doesn't have positive results (i.e., potential curvature).

In [ ]:
# QQ-plot:
ggplot(data=hills_fit_fort, aes(sample = .stdresid)) +
  stat_qq(geom="point") +
  geom_abline() +
  coord_equal()


Big red flags!
- Residuals do not follow the diagonal line.
- Strong evidence against normality assumptions.

In [ ]:
# Histogram:
ggplot(data=hills_fit_fort, aes(x=.stdresid)) +
  geom_histogram(fill="grey80", binwidth=0.5,
                 aes(y=after_stat(density))) +
  stat_function(fun=dnorm, xlim=c(-3,3)) +
  geom_rug()


And now for more bad news!
- Too much mass in the centre.
- A clear outlier.

What if we remove that outlier?

In [ ]:
# identify the outliers
# unpack this carefully in the lecture!
(
outliers <- hills_fit_fort %>%
  arrange(desc(.stdresid)) %>%
  slice(1:2) %>%
  row.names()
)

In [ ]:
# create a reduced data frame excluding the outlier
hills_no_outliers <- hills %>%
  filter(!(row.names(hills) %in% outliers))

# fit a model to the new dataset
hills_fit_no_outliers <-
  lm(data = hills_no_outliers, time ~ dist + climb)

# Now check the QQ plot:
hills_fit_fort_no_outliers <- fortify(hills_fit_no_outliers)
ggplot(data=hills_fit_fort_no_outliers,
       aes(sample = .stdresid)) +
  stat_qq(geom="point") +
  geom_abline() +
  coord_equal()


In [ ]:
# The histogram?
ggplot(data=hills_fit_fort_no_outliers, aes(x=.stdresid)) +
  geom_histogram(fill="grey80", binwidth=0.5,
                 aes(y=..density..)) +
  stat_function(fun=dnorm, xlim=c(-3,3)) +
  geom_rug()

Much more like it!

### 5. Interpret model.


- Make predictions for useful scenarios.
- Summarise analysis with regards to need/question.
- Compare to similar studies.
- State conclusions you have come to, using analysis results as evidence.


#### Predictions

In [ ]:
# Can't visualise predictions with a line, because there are 3 "dimensions" now.
# Use a plane instead, with different combinations of dist and climb:

# get the limits to use for the plane
range(hills_no_outliers$dist)
range(hills_no_outliers$climb)

In [ ]:
# create a grid to hold values over the plane
dist_new <- seq(from=2, to=28, by=0.1)
climb_new <- seq(from=300, to=5200, by=10)
hills_newdata <- expand.grid(dist=dist_new,
                             climb=climb_new)

# fill the grid with predictions
hills_newdata$time <- predict(hills_fit_no_outliers, hills_newdata)

# plot the plane
hills_pred_ggplot <- ggplot(data = hills_newdata, aes(x=dist, y=climb)) +
  geom_raster(aes(fill=time)) +
  scale_fill_gradient(low="red", high="blue") +
  xlab("Distance (miles)") +
  ylab("Climb (feet)") +
  theme(legend.position="right")
hills_pred_ggplot

- Increasing climb height increases the time taken.
- Increasing the distance run increases the time taken.
- Both an increase in climb height and distance run increases the overall time.


#### Summarise


- Can see dist and climb play an important role in predicting time.
- So, we fit a linear regression model with two explanatory variables.
- Need to be careful about meeting our assumptions.
- Justify and omit outliers with caution.


## Slide 17

The `whiteside` data set from the `MASS` library contains weekly gas consumption and average outdoor temperature before and after installing insulation.
1. Analyse gas consumption as a model insulation and temperature, considering interactions.
2. Consider the same parameters, this time fitting a model with no intercept.


In [ ]:
# grab the whiteside data set
whiteside <- MASS::whiteside

# Visualise weekly gas consumption and average outdoor temperature before and
# after installing insulation.
gg.white <- ggplot(data=whiteside, aes(x=Temp, y=Gas)) +
  geom_point() + geom_smooth(method="lm") +
  facet_wrap( ~ Insul) +
  xlab("Temperature (Celsius)") +
  ylab(expression(Gas~consumption~(10^{3}~cubic~feet)))
gg.white

- Gas consumption lower overall when insulation installed.
- Maybe the slope of the consumption-temperature line is different before and
  after insulation is installed.
- What effect does installing insulation have on gas consumption?

### Fit the first model:

In [ ]:
summary(
lm.gas <- lm(data=whiteside, Gas ~ Temp * Insul)
)

This is equivalent to:

$$
Gas_i = β_0 + β_1 × Temp_i + β_2 × Ι(Insul_i == \text{After}) +
        β_3 × Temp_i × Ι(Insul_i == \text{After}) + ε_i.
$$

Where $Ι()$ is an indicator function (see week 10).


#### Aside

When you look at the results, you will see a colon, `:`, used in place of the
typical multiplication sign for interaction terms in your model summary.

This is because `R` uses the multiplication sign (`*`) in the linear model to
represent the interaction term and the individual terms, i.e.,
`lm(y ~ x*z)` gives:

$$
y_i = β_0 + β_1 x_i + β_2 z_i + β_3 x_i z_i + ε_i
$$

Where `lm(y ~ x:z)` gives:
$$
y_i = β_0 + β_1 x_i z_i + ε_i
$$

We never use the latter in practice.


### Interpret the first model:


In [ ]:
gas.coef <- tidy(lm.gas, conf.int=T) %>%
  rename(`2.5%` = conf.low,
         `97.5%` = conf.high)
gas.coef

Model output now shows contribution to gas consumption as:
- `(Intercept)`: Baseline of temperature at 0 degrees before insulation.
- `Temp`: Effect of temperature.
- `InsulAfter`: Effect of installing insulation.
- `Temp:InsulAfter`: Additional effect of temperature after insulation is
  installed.


In [ ]:
glance(lm.gas)


- All model terms significant.
- 92.35% of variability explained.
- Baseline gas consumption (ignoring effect of temperature) lower after insulation installed.
- Effect of InsulAfter is -2.13, with 95% CI (-2.4914, -1.7686).
- Effect of Temp is -0.3932, additional effect of Temp:InsulAfter is 0.1153 which means total effect of Temp post-installation is -0.2779.
- Gas consumption not as sensitive to temperature changes because slope of line is *less* negative after installing insulation. (i.e. magnitude is smaller, closer to 0)


### Fit the second model (no intercept):

In [ ]:
summary(lm.gas2 <- lm(data=whiteside, Gas ~ Insul + Temp:Insul - 1))


- Model is a little more interpretable by specifying the model differently.
- Now, we only look at the effect of temperature through its interaction with insulation.


In [ ]:
gas.coef2 <- lm.gas2 %>%
  tidy(conf.int=T) %>%
  rename(`2.5%` = conf.low,
         `97.5%` = conf.high)
gas.coef2

No longer making comparisons against a baseline
- Baseline in previous model was `Insul = Before`, estimate still same
- `InsulAfter` is no longer the difference between before and after installation
- `InsulBefore:Temp` is the effect of temperature before installation
- `InsulAfter:Temp` is the effect of temperature after installation

These two models are functionally the same.
- just specified differently
- explain the exact same amount of variability
- Remember that R-squared will be incorrectly reported in model with no intercept.


In [ ]:
glance(lm.gas)
glance(lm.gas2)

Key takeaways:
- Installing insulation dropped gas consumption at 0 degrees Celsius.
- Installing insulation made gas consumption less sensitive to changes in temperature.


## Slide 19 - Basic Principles – Model Selection

Consider the `whiteside` gas data set again. Assume that installing insulation only changes the intercept, i.e., the temperature-consumption relationship still has the same slope. This becomes a model with parallel lines, i.e., no interaction between variables.

$$
Gas_i = β_0 + β_1 Temp_i + β_2 Ι(Insul_i = \text{After}) + ε_i
$$

Does the previous model (with the interaction term) account for more variability in gas consumption?


In [ ]:
# Our existing model:
lm.gas <- lm(data = whiteside, Gas ~ Temp * Insul)


In [ ]:
# Our parallel model:
lm.gas.parallel <- lm(data = whiteside, Gas ~ Temp + Insul)

In [ ]:
# Visualisation:
# grid
newdata.gas <- expand.grid(Temp = seq(-1,10,by=1),
                           Insul = c("Before", "After"))
# full model
pred.gas <- predict(lm.gas, newdata.gas) %>%
  data.frame(fit = .) %>%
  mutate(Model = "Full interaction") %>%
  bind_cols(newdata.gas)
# reduced model
pred.gas.parallel <- predict(lm.gas.parallel, newdata.gas) %>%
  data.frame(fit = .) %>%
  mutate(Model = "Parallel") %>%
  bind_cols(newdata.gas)
# plot
bind_rows(pred.gas, pred.gas.parallel) %>%
  ggplot(data=., aes(x=Temp, y=fit)) +
  geom_line(aes(group=Insul, color=Insul)) +
  facet_wrap( ~ Model, nrow=1) +
  geom_point(data=whiteside, aes(color=Insul, y=Gas)) +
  xlab("Temperature (Celsius)") +
  ylab(expression(Gas~consumption~(10^{3}~cubic~feet))) +
  theme(legend.position="right")

In [ ]:
# Comparison of R-squared values:
glance(lm.gas)
glance(lm.gas.parallel)


In [ ]:
# Both have Adj R-squared above 0.9
# Small increase when adding the interaction.


In [ ]:
# Running ANOVA for model comparison:
anova(lm.gas.parallel, lm.gas)


Model 1 nested inside Model 2.
- $p < 0.05$ so reject $H_0$.
- Rejecting $H_0$ implies that interaction model accounts for more variability.
- Interaction model is better based on this analysis.